In [1]:
import os

import numpy as np
import pandas as pd
from pandas import DataFrame
from scipy.stats.mstats import winsorize
from gmpy2 import mpz

def get_parent(path=os.getcwd(), levels=1):
    common = path
    # Using for loop for getting starting point required for
    for i in range(levels + 1):
        # Starting point
        common = os.path.dirname(common)
    return os.path.abspath(common)

#from codes.shared_library.utils import POOL_INFO, UNISWAP_NFT_MANAGER, get_parent, \
#    UNISWAP_MIGRATOR

In [2]:
data_folder_path = os.path.join(get_parent(), "data")
sc_verification_data = pd.read_csv(os.path.join(data_folder_path, "01_original_data", "sc_ownership", 'sc_verified_data.csv'))
sc_with_token_transfers = pd.read_csv(os.path.join(data_folder_path, "01_original_data", "sc_ownership", '0911_sc_ownership_try_second_time.csv'))

In [3]:
#ownership_percentage = pd.read_csv("ownership_result_test.csv")
sc_with_token_transfers["RAW_AMOUNT"] = sc_with_token_transfers["RAW_AMOUNT"].astype(float)


In [4]:
sc_verification_dates_data = pd.read_csv(os.path.join(data_folder_path, "01_original_data", "sc_ownership", 'verification_dates.csv'))
sc_contract_data = pd.read_csv(os.path.join(data_folder_path, "01_original_data", "sc_ownership", 'sc_contract_data.csv'))

In [5]:
all_zero_address = '0x0000000000000000000000000000000000000000'
cond_wired = (sc_with_token_transfers['FROM_ADDRESS'] != all_zero_address) & (sc_with_token_transfers['TO_ADDRESS'] != all_zero_address)

In [12]:
sc_with_token_transfers["RAW_AMOUNT_PRECISE"] = sc_with_token_transfers["RAW_AMOUNT_PRECISE"].astype(float)
sc_with_token_transfers.dropna(inplace=True)

In [22]:
len(contract_addresses)

141

In [25]:
# Convert the BLOCK_TIMESTAMP to a datetime object
df = sc_with_token_transfers.copy()
df.sort_values(by=['CONTRACT_ADDRESS', 'BLOCK_TIMESTAMP'], inplace=True)
df['BLOCK_TIMESTAMP'] = pd.to_datetime(df['BLOCK_TIMESTAMP'])
user_addresses = set(df['FROM_ADDRESS']).union(set(df['TO_ADDRESS']))
contract_addresses = set(df['CONTRACT_ADDRESS'])

# Create a dictionary to store the balances for each address
balances = {}

# Create a dictionary to store the total supply for each token
total_supplies = {}

# Create a new DataFrame to store the daily balances
daily_balances = pd.DataFrame(columns=['ADDRESS', 'CONTRACT_ADDRESS', 'BALANCE', 'DATE'])
max_date = df['BLOCK_TIMESTAMP'].max()
# Iterate over each address and contract_address combination
for contract_address in contract_addresses:
    filtered_df = df[df['CONTRACT_ADDRESS'] == contract_address][['BLOCK_NUMBER', 'BLOCK_TIMESTAMP', 'TX_HASH', 'EVENT_INDEX', 
                                                                  'CONTRACT_ADDRESS', 'FROM_ADDRESS', 'TO_ADDRESS', 'RAW_AMOUNT', 'RAW_AMOUNT_PRECISE']]
    user_addresses = set(filtered_df['FROM_ADDRESS']).union(set(filtered_df['TO_ADDRESS']))
    for address in user_addresses:
        if address == all_zero_address:
            continue

        # Filter the transactions for the current address and contract_address
        filtered_df_from_this_address = filtered_df[filtered_df['FROM_ADDRESS'] == address].copy().reset_index(drop=True)
        filtered_df_to_this_address = filtered_df[filtered_df['TO_ADDRESS'] == address].copy().reset_index(drop=True)
        if filtered_df_from_this_address.shape[0] + filtered_df_to_this_address.shape[0] == 0:
            continue
        # if transferred from this address then amount should be minus
        filtered_df_from_this_address['RAW_AMOUNT_PRECISE'] *= -1
        result_df = pd.concat([filtered_df_from_this_address, filtered_df_to_this_address])
        result_df.sort_values(by=['BLOCK_TIMESTAMP', 'EVENT_INDEX'], inplace=True)
        sum_total_at_the_end = result_df['RAW_AMOUNT_PRECISE'].sum()
        if sum_total_at_the_end < 0:
            continue
        elif sum_total_at_the_end > 0:
            fake_data = result_df.tail(1).copy()
            fake_data["BLOCK_TIMESTAMP"] = max_date
            fake_data["RAW_AMOUNT_PRECISE"] = 0
            result_df = pd.concat([result_df, fake_data], axis=0, ignore_index=True)

        # Resample the transactions to daily frequency and calculate the balance
        daily_df = result_df.resample('D', on='BLOCK_TIMESTAMP').agg({'RAW_AMOUNT_PRECISE': 'sum'}).reset_index()
        daily_df['BALANCE'] = daily_df['RAW_AMOUNT_PRECISE'].cumsum()
        daily_df['ADDRESS'] = address
        daily_df['CONTRACT_ADDRESS'] = contract_address
        daily_df['DATE'] = daily_df['BLOCK_TIMESTAMP'].dt.date

        # Append the daily balances to the result DataFrame
        daily_balances = pd.concat([daily_balances, daily_df[['ADDRESS', 'CONTRACT_ADDRESS', 'BALANCE', 'DATE']]], ignore_index=True)

In [28]:
daily_balances.shape

(1896007, 4)

In [29]:
wrong_address_ids = daily_balances[daily_balances["BALANCE"] < 0]["ADDRESS"].unique()
correct_daily_df = daily_balances[~daily_balances["ADDRESS"].isin(wrong_address_ids)].copy()

In [30]:
correct_daily_df

,ADDRESS,CONTRACT_ADDRESS,BALANCE,DATE
0,0xbdfa4f4492dd7b7cf211209c4791af8d52bf5c50,0x9683d433621a83aa7dd290106e1da85251317f55,6.618387e+21,2022-01-13
1,0xbdfa4f4492dd7b7cf211209c4791af8d52bf5c50,0x9683d433621a83aa7dd290106e1da85251317f55,6.618387e+21,2022-01-14
2,0xbdfa4f4492dd7b7cf211209c4791af8d52bf5c50,0x9683d433621a83aa7dd290106e1da85251317f55,6.618387e+21,2022-01-15
3,0xbdfa4f4492dd7b7cf211209c4791af8d52bf5c50,0x9683d433621a83aa7dd290106e1da85251317f55,6.618387e+21,2022-01-16
4,0xbdfa4f4492dd7b7cf211209c4791af8d52bf5c50,0x9683d433621a83aa7dd290106e1da85251317f55,6.618387e+21,2022-01-17
...,...,...,...,...
1896002,0xcf3f5b3fc1c4abefffadba2bd18f533ffa2a6724,0xb6f1a307a0c9a03c67f8d646809e472014a3ee65,8.930000e+24,2022-12-29
1896003,0xcf3f5b3fc1c4abefffadba2bd18f533ffa2a6724,0xb6f1a307a0c9a03c67f8d646809e472014a3ee65,8.930000e+24,2022-12-30
1896004,0xcf3f5b3fc1c4abefffadba2bd18f533ffa2a6724,0xb6f1a307a0c9a03c67f8d646809e472014a3ee65,8.930000e+24,2022-12-31
1896005,0x272163e3f1735309609f22ccc8c7e330f961bfa6,0x4f65c0f7285d647e3b156c73cc5ebdc393d5aad6,0.000000e+00,2021-06-07


In [31]:
daily_balances_each_contract = correct_daily_df.groupby(["DATE", "CONTRACT_ADDRESS"])["BALANCE"].sum().reset_index()

In [32]:
daily_balances_each_contract = daily_balances_each_contract[daily_balances_each_contract["BALANCE"] > 0]
daily_balances_each_contract.rename(columns={"BALANCE": "CONTRACT_BALANCE"}, inplace=True)

In [33]:
correct_daily_df_with_contract_balances = correct_daily_df.merge(daily_balances_each_contract, on=["DATE", "CONTRACT_ADDRESS"], how='left')

In [34]:
sum(correct_daily_df_with_contract_balances["CONTRACT_BALANCE"].isna())

201

In [35]:
correct_daily_df_with_contract_balances = correct_daily_df_with_contract_balances[~correct_daily_df_with_contract_balances.isna()]

In [36]:
correct_daily_df_with_contract_balances["percentage_ownership"] = correct_daily_df_with_contract_balances["BALANCE"] / correct_daily_df_with_contract_balances["CONTRACT_BALANCE"]

In [37]:
correct_daily_df_with_contract_balances["week"] = pd.to_datetime(correct_daily_df_with_contract_balances['DATE']).dt.to_period('W-SAT').dt.start_time

In [38]:
correct_daily_df_with_contract_balances["week"] = correct_daily_df_with_contract_balances["week"].astype(str)

In [39]:
correct_daily_df_with_contract_balances.groupby(['week', 'ADDRESS', 'CONTRACT_ADDRESS'])["percentage_ownership"].mean().reset_index().groupby(["week", "CONTRACT_ADDRESS"])["percentage_ownership"].sum().reset_index().describe()

,percentage_ownership
count,5947.000000
mean,1.006737
std,0.090274
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.077364


In [40]:
ownership_percentage = correct_daily_df_with_contract_balances.sort_values(by=['CONTRACT_ADDRESS', 'ADDRESS', 'DATE']).groupby(['week', 'ADDRESS', 'CONTRACT_ADDRESS'])["percentage_ownership"].last().reset_index()

In [41]:
ownership_percentage

,week,ADDRESS,CONTRACT_ADDRESS,percentage_ownership
0,2021-05-02,0x000f4432a40560bbff1b581a8b7aded8dab80026,0xb52f322f7534d60807700bd8414d3c498d4cef52,0.009088
1,2021-05-02,0x1626d1683ab6989960a7a86e94c86d783414869d,0xb52f322f7534d60807700bd8414d3c498d4cef52,0.005846
2,2021-05-02,0x178cfe4e55fd5720a1ede7f3f3f7f096c678a648,0xb52f322f7534d60807700bd8414d3c498d4cef52,0.000356
3,2021-05-02,0x279a3fdbd6d6252afc5c422439c7d7859a51a05e,0xb52f322f7534d60807700bd8414d3c498d4cef52,0.000167
4,2021-05-02,0x297946c26171008ba8c0e5642814b5fe6b842ab7,0xb52f322f7534d60807700bd8414d3c498d4cef52,0.074318
...,...,...,...,...
274880,2022-12-25,0xff703ebb19c1a4e27ba14e8d7538fc113ca5f906,0x9bf7b46c7ad5ab62034e9349ab912c0345164322,0.001491
274881,2022-12-25,0xffc8dae3fb28dc6224ae07fc0733001c9e1949e0,0x716bd8a7f8a44b010969a1825ae5658e7a18630d,0.070539
274882,2022-12-25,0xfff33c0bde72f6472f1d185166b7cbfcc3e9e150,0xd63b340f6e9cccf0c997c83c8d036fa53b113546,0.004525
274883,2022-12-25,0xfffa09b517354968d378573c9a31d1586fd8792d,0x9bf7b46c7ad5ab62034e9349ab912c0345164322,0.000232


In [42]:
ownership_percentage[ownership_percentage["percentage_ownership"] > 0].rename(columns={"ADDRESS":"liquidity_provider", "CONTRACT_ADDRESS": "nf_position_manager_address"}).to_csv("ownership_result0916.csv", index=False)

In [146]:
ownership_percentage.rename(columns={"ADDRESS":"liquidity_provider", "CONTRACT_ADDRESS": "nf_position_manager_address"}).groupby(["liquidity_provider"]).apply(lambda x: x[].unique().shape)

AttributeError: 'DataFrame' object has no attribute 'unique'

In [43]:
ownership_percentage[ownership_percentage["percentage_ownership"] > 0].shape

(270399, 4)

In [139]:
189833/4266

44.49906235349273

In [1]:
ownership_percentage

NameError: name 'ownership_percentage' is not defined

In [148]:
ownership_percentage.rename(columns={"ADDRESS":"liquidity_provider", "CONTRACT_ADDRESS": "nf_position_manager_address"}).to_csv("0613_ownership_percentage.csv", index=False)